## Data Ingestion

In [1]:
from langchain_community.document_loaders import WebBaseLoader

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [2]:
from src.constants import traning_pipline
url=traning_pipline.DATA_INGESTION_DATA_URL

In [3]:
#url='https://en.wikipedia.org/wiki/DeepSeek'
loder=WebBaseLoader(url)
docs=loder.load()

In [4]:
all_text=''
filename=r'D:\assamement-project\Wikipedia-RAG-Based-Application\data\data.txt'

In [5]:
import os
dir_path = os.path.dirname(filename)
os.makedirs(dir_path, exist_ok=True)

In [6]:
with open(filename, "w", encoding="utf-8") as f:
   f.write(all_text)

In [7]:
len(all_text)

0

In [8]:
all_text

''

# Chunking Documents

In [9]:
with open(filename,'r',encoding='utf-8') as f:
    raw_text= f.read()

In [10]:
import re
def preprocess_text(text: str) -> str:
    text = text.lower()  # Convert to lowercase
    text = re.sub(r"[^a-zA-Z0-9\n]", " ", text)  # Remove special characters
    text = re.sub(r"([a-z])([A-Z])", r"\1 \2", text)  # Add spaces between camelCase words
    text = re.sub(r"(\d+)", r" \1 ", text)  # Add spaces around numbers
    text = re.sub(r"\s+", " ", text).strip()  # Remove extra spaces
    return text

In [11]:
raw_text

''

In [12]:
cleaned_text=preprocess_text(all_text)

In [13]:
cleaned_text

''

In [14]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=50
         )

In [15]:
# split doc
chunks= text_splitter.split_text(cleaned_text)

In [16]:
chunks[1]

IndexError: list index out of range

In [80]:
len(chunks)

90

In [82]:
chunks[0]

'https en wikipedia org wiki deepseek deepseek wikipedia jump to content main menu main menu move to sidebar hide navigation main pagecontentscurrent eventsrandom articleabout wikipediacontact us contribute helplearn to editcommunity portalrecent changesupload file search search appearance donate create account log in personal tools donate create account log in pages for logged out editors learn more contributionstalk contents move to sidebar hide top 1 background 2 release history toggle release'

In [166]:
filename=r'D:\assamement-project\Wikipedia-RAG-Based-Application\expariment\chunk\data.json'
dir_path = os.path.dirname(filename)
os.makedirs(dir_path, exist_ok=True)


In [167]:
import json


with open(filename, "w", encoding="utf-8") as f:
    json.dump(chunks, f, indent=4, ensure_ascii=False)  # Save list as JSON


In [168]:
with open(filename, "r", encoding="utf-8") as f:
    chunks = json.load(f)  # Load list from JSON

print(chunks)


['https en wikipedia org wiki deepseek deepseek wikipedia jump to content main menu main menu move to sidebar hide navigation main pagecontentscurrent eventsrandom articleabout wikipediacontact us contribute helplearn to editcommunity portalrecent changesupload file search search appearance donate create account log in personal tools donate create account log in pages for logged out editors learn more contributionstalk contents move to sidebar hide top 1 background 2 release history toggle release', 'top 1 background 2 release history toggle release history subsection 2 1 deepseek llm 2 2 v 2 2 3 v 3 2 4 r 1 3 assessment and reactions 4 concerns toggle concerns subsection 4 1 censorship 4 2 security and privacy 5 see also 6 notes 7 references 8 external links toggle the table of contents deepseek 47 languages afrikaans aragon s catal e tinadanskdeutsch espa olesperanto fran aisfulfuldegaeilge bahasa indonesiaitaliano kiswahilimagyar nederlands o zbekcha polskiportugu sqaraqalpaqsharom 

In [169]:
chunks[0]

'https en wikipedia org wiki deepseek deepseek wikipedia jump to content main menu main menu move to sidebar hide navigation main pagecontentscurrent eventsrandom articleabout wikipediacontact us contribute helplearn to editcommunity portalrecent changesupload file search search appearance donate create account log in personal tools donate create account log in pages for logged out editors learn more contributionstalk contents move to sidebar hide top 1 background 2 release history toggle release'

# vectordb

In [110]:
from langchain_google_genai.embeddings import GoogleGenerativeAIEmbeddings

embeddings=GoogleGenerativeAIEmbeddings(
   google_api_key=os.getenv('GEMINI_API_KEY'),
   model="models/embedding-001"
)

In [112]:
d=embeddings.embed_query('how are you')
len(d)

768

In [114]:
d=embeddings.embed_query(' jdsv udpv vsouvepu \buj\\')
len(d)

768

In [134]:
print(os.getenv('PINCONE_API_KEY'))

None


In [142]:
import time
from pinecone import Pinecone,ServerlessSpec

pinecone_api_key = os.environ.get("PINECONE_API_KEY")

pc = Pinecone(api_key='pcsk_2e9E7E_PTURkSdk3ktVBRXriXwcd8XmbcTD8dqKhcqNDbRomkKcVA6QexdHcrCcdxEP4dD')
index_name = "rag" 

existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=3072,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )
    while not pc.describe_index(index_name).status["ready"]:
        time.sleep(1)

index = pc.Index(index_name)

In [143]:
from langchain_pinecone import PineconeVectorStore
vector_store = PineconeVectorStore(index=index, embedding=embeddings)

In [145]:
vector_store.add_texts(texts=chunks)

['91430d65-3ae8-49b6-86fb-11be62e2955e',
 'ef22e4cf-3b66-4c2f-b2f6-9911a007ace3',
 '7fdd630f-8bab-4b97-a7fd-d0d14e7eb475',
 'de94371f-6299-4d28-81b0-f12b3dd146c5',
 'fa25e20d-552f-4081-a6a7-40bf5ad36de0',
 'fdaef7f9-a0ad-4789-ba90-2c6d3a8bdee6',
 'b132ca4d-88b1-49b5-9596-f31b66ba4c99',
 'de336e36-1f92-4730-8934-0364e8df2770',
 '3f169102-3bb9-4af1-beca-ea2db73a71be',
 '669ea975-098b-4534-b1d7-8b2c91a7c60f',
 'cdb8ca66-5b73-44e2-b81c-882eaa5db020',
 'ac65f7df-8ee7-4f97-95cc-2a0a6e7b6d1a',
 'fff35f81-fa0b-4e85-b997-dc86acd0a120',
 '5b6d6e15-74eb-45e6-bb55-c8746f70a4fa',
 '0c276a49-a454-4e64-986a-051be4e0f671',
 'd1d7556c-ee91-4252-894c-08ff2a96212d',
 'd4d4fbd0-583b-4d4a-ab9e-51e62a278cb2',
 'c7c0df9b-1621-42f8-a5df-baccbfce1481',
 'f1b5624c-363e-499f-a369-b1ab1206d8e4',
 '05eec6c1-12d4-405c-9d6d-0d3d520b7c86',
 'f0d9c070-171b-4937-9fc5-82c1756d2e0d',
 '03f656f1-1eb2-451a-87c1-77c517cdc2db',
 '95c9b040-1c64-48bf-8135-6b7da509c6fa',
 '9aebb245-c788-474d-a399-3c5eb65911f6',
 '10368cb3-90f4-

In [146]:
query='what is deepseek model'

similarity_search=vector_store.similarity_search(query)

In [149]:
similarity_search[0].page_content

'for math problems was computed by comparing with the ground truth label the reward for code problems was generated by a reward model trained to predict whether a program would pass the unit tests deepseek v 2 5 was released in september and updated in december it was made by combining deepseek v 2 chat and deepseek coder v 2 instruct 36 v 3 in december 2024 they released a base model deepseek v 3 base and a chat model deepseek v 3 the model architecture is essentially the same as v 2 they were'

In [171]:
from langchain_community.vectorstores import FAISS
db= FAISS.from_texts(chunks,embeddings)

In [176]:
db.index.ntotal

90

In [ ]:
db.save_local('vectordb/Faiss_db')

## Retrival

In [152]:
from langchain_google_genai import GoogleGenerativeAI
from langchain.chains import RetrievalQA

llm=GoogleGenerativeAI(
   api_key=os.getenv('GEMINI_API_KEY'),
   model='gemini-pro',
   top_k=3
)

In [153]:
qa= RetrievalQA.from_chain_type(
    llm=llm,
    chain_type='stuff',
    retriever=vector_store.as_retriever()
)

In [154]:
qa.invoke('Tell me about Deepseak models')

{'query': 'Tell me about Deepseak models',
 'result': 'Deepseak models are a series of large language models developed by Deepseak AI. The models are trained on a massive dataset of English and Chinese text, and they are designed to perform a variety of natural language processing tasks, such as text generation, translation, and question answering. Deepseak models have been used to develop a variety of applications, including chatbots, search engines, and language translation tools.'}

In [155]:
qa.invoke('Tell me about Deepseak model architechure')

{'query': 'Tell me about Deepseak model architechure',
 'result': 'v2 models were significantly modified from the deepseek llm series, they changed the standard attention mechanism by a low rank approximation called multi head latent attention mla and used the mixture of experts moe variant previously published in january compared'}

In [172]:
## fiaas db
qa= RetrievalQA.from_chain_type(
    llm=llm,
    chain_type='stuff',
    retriever=db.as_retriever()
)

In [173]:
qa.invoke('Tell me about Deepseak models')

{'query': 'Tell me about Deepseak models',
 'result': 'Deepseak models are a series of large language models (LLMs) developed by Chinese software engineers. The models are trained on a massive dataset of English and Chinese text, and they are capable of performing a wide range of natural language processing tasks, such as text generation, translation, question answering, and dialogue. Deepseak models have been used to develop a variety of applications, including chatbots, search engines, and language learning tools.'}

In [174]:
qa.invoke('Tell me about Deepseak model architechure')

{'query': 'Tell me about Deepseak model architechure',
 'result': 'Deepseak v2 models were significantly modified from the DeepSeek LLM series. They changed the standard attention mechanism to a low rank approximation called Multi-Head Latent Attention (MLA) and used the Mixture of Experts (MoE) variant published in January. DeepSeek v3 models have essentially the same architecture as v2, but are trained differently.'}

In [180]:
new_db= FAISS.load_local('vectordb/Faiss_db',embeddings,allow_dangerous_deserialization=True)

In [182]:
new_db.similarity_search('Tell me about Deepseak model architechure')

[Document(id='5b2895cc-825d-4f32-9c21-25f01f80e19d', metadata={}, page_content='the two v 2 lite models were smaller and trained similarly though deepseek v 2 lite chat only underwent sft not rl they trained the lite version to help further research and development on mla and deepseekmoe 30 architecturally the v 2 models were significantly modified from the deepseek llm series they changed the standard attention mechanism by a low rank approximation called multi head latent attention mla and used the mixture of experts moe variant previously published in january compared'),
 Document(id='0087d14b-1988-4178-b987-216b5b67ff5f', metadata={}, page_content='models and training details open source allowing its code to be freely available for use modification viewing and designing documents for building purposes 13 the company reportedly vigorously recruits young a i researchers from top chinese universities 8 and hires from outside the computer science field to diversify its models knowledge